# Presets

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
pd.set_option('display.max_rows', 100)

# Functions

In [2]:
def get_sheet(ticker):
    df = yf.Ticker(ticker).quarterly_balance_sheet.reset_index().iloc[:, [0, 1]]
    df.columns = ['col1', 'col2']
    df['col1'] = df['col1'].str.lower().str.replace(' ', '_')
    df = pd.DataFrame([df['col2'].tolist()], columns=df['col1'].tolist())
    return(df)

def get_cf(ticker):
    df = yf.Ticker(ticker).quarterly_cash_flow.reset_index().iloc[:, [0, 1]]
    df.columns = ['col1', 'col2']
    df['col1'] = df['col1'].str.lower().str.replace(' ', '_')
    df = pd.DataFrame([df['col2'].tolist()], columns=df['col1'].tolist())
    return(df)

def get_inc(ticker):
    df = yf.Ticker(ticker).quarterly_income_stmt.reset_index().iloc[:, [0, 1]]
    df.columns = ['col1', 'col2']
    df['col1'] = df['col1'].str.lower().str.replace(' ', '_')
    df = pd.DataFrame([df['col2'].tolist()], columns=df['col1'].tolist())
    return(df)

def get_dividents(ticker):
    df = pd.DataFrame({
        'dividends': [yf.Ticker(ticker).get_dividends().to_frame().reset_index().query('Date >= "2023-01-01 00:00:00-04:00"')['Dividends'].sum()]
    })
    return(df)

def get_book_data(ticker):
    df = pd.concat([get_inc(ticker), get_cf(ticker), get_sheet(ticker), get_dividents(ticker)], axis=1)

    columns_to_check = [
        'change_in_other_current_liabilities', 'cash_dividends_paid', 'repurchase_of_capital_stock', 'repayment_of_debt', 'issuance_of_debt', 'research_and_development', 'total_debt', 'operating_expense', 'change_in_inventory',
        'inventory', 'capital_expenditure', 'gross_profit', 'operating_income', 'net_ppe', 'current_liabilities', 'current_assets', 'operating_cash_flow', 'net_income_from_continuing_operations', 'free_cash_flow', 'change_in_working_capital',
        'cash_and_cash_equivalents', 'receivables', 'total_assets', 'to_extract'
    ]
    for column in columns_to_check:
        if column not in df.columns:
            df[column] = 0

    df['rdi_perc'] = df['research_and_development']/df['operating_expense']
    df['gross_profit_margin'] = df['gross_profit']/df['total_revenue']# profitability of a company's core operations as a percentage of total revenue
    df['operating_income_perc'] = df['operating_income']/df['gross_profit']
    df['eps'] = df['basic_eps']
    df['operating_cash_flow_to_sales'] = df['operating_cash_flow'] / df['net_income_from_continuing_operations']
    df['free_cash_flow_to_net_income'] = df['free_cash_flow'] / df['net_income_from_continuing_operations']
    df['cash_flow_to_capex'] = df['operating_cash_flow'] / df['capital_expenditure']
    df['cash_dividends_paid_ratio'] = df['cash_dividends_paid'] / df['net_income_from_continuing_operations']
    df['debt_to_equity_ratio'] = df['repayment_of_debt'] / (df['issuance_of_debt'] + df['repurchase_of_capital_stock'])
    df['current_ratio'] = df['change_in_working_capital'] / df['change_in_other_current_liabilities']
    df['quick_ratio'] = (df['change_in_working_capital'] - df['change_in_inventory']) / df['change_in_other_current_liabilities']
    df['to_extract'] = (df['net_ppe'] + df['cash_and_cash_equivalents'] + df['receivables'] + df['inventory'])
    df['total_equity'] = df['total_assets'] - df['total_debt'] - df['current_liabilities'] - df['to_extract']
    df['total_liabilities'] = df['total_assets'] - df['total_equity']   
    df['debt_to_equity_ratio'] = df['total_liabilities'] / df['total_equity']
    df['current_ratio'] = df['current_assets'] / df['current_liabilities']
    df['quick_ratio'] = (df['cash_and_cash_equivalents'] + df['receivables'] + df['inventory']) / df['current_liabilities']
    df['return_on_equity'] = df['net_income_from_continuing_operations'] / df['total_equity']
    df['return_on_assets'] = df['net_income_from_continuing_operations'] / df['total_assets']
    df['asset_turnover'] = df['total_revenue'] / ((df['total_assets'] + df['total_assets']) / 1)  # Assuming only two periods

    cols_to_keep = [
        'operating_cash_flow_to_sales', 'free_cash_flow_to_net_income', 'cash_flow_to_capex', 'cash_dividends_paid_ratio', 'debt_to_equity_ratio', 'current_ratio', 'quick_ratio',
        'gross_profit_margin', 'operating_income_perc', 'rdi_perc', 'eps', 'asset_turnover', 'return_on_assets', 'return_on_equity', 'dividends'
    ]
    df = df.loc[:, cols_to_keep]
    return(df)

In [3]:
def get_prices(ticker):
    df = pd.DataFrame(dict(
        lower_price_ratio = [yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['Low'].min()/yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['Adj Close'].mean()],
        upper_price_ratio = [yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['High'].max()/yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['Adj Close'].mean()],
        std_to_mean_price_perc = [yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['Adj Close'].std()/yf.download(ticker, start='2023-01-01', end='2023-12-31', progress=False)['Adj Close'].mean()]
    ))
    return(df)

# Get yahoo data for each stock

In [4]:
stock_dict = {
    166: 'KHC', 121: 'KDP', 105: 'MDLZ', 151: 'CSCO', 170: 'HOLX', 0: 'MNST', 65: 'EXC', 109: 'CSX', 123: 'GILD', 198: 'CMCSA', 131: 'INCY', 21: 'SSNC', 148: 'XEL',
    38: 'ATVIX', 30: 'LNT', 63: 'LKQ', 24: 'AKAM', 130: 'SBUX', 120: 'FOXA', 195: 'AEP', 53: 'EBAY', 81: 'SGEN', 47: 'TXRH', 154: 'DBX', 160: 'PEP', 55: 'FAST',
    37: 'ADP', 90: 'FFIV', 186: 'CTSH', 187: 'EA', 76: 'HAS', 117: 'AGNC', 134: 'VTRS', 3: 'HON', 165: 'HST', 97: 'NBIX', 145: 'CG', 25: 'EXPD', 68: 'PAYX', 52: 'CINF',
    112: 'LSTR', 181: 'JBHT', 28: 'FANG', 43: 'JKHY', 12: 'AMGN', 149: 'VRSK', 144: 'PCAR', 192: 'PTC', 153: 'HTZ', 175: 'GOOGL', 189: 'CSGP', 116: 'CDW', 35: 'ROST',
    46: 'TECH', 164: 'CPRT', 44: 'CME', 146: 'MIDD', 125: 'UTHR', 171: 'TROW', 73: 'GEN', 196: 'XRAY', 9: 'PFG', 199: 'BKR', 193: 'PYPL', 106: 'TRMB', 122: 'AAL', 4: 'MAR', 
    176: 'FTNT', 2: 'AXON', 167: 'CHRW', 91: 'DOCU', 128: 'WDC', 152: 'SBAC', 155: 'CHK', 26: 'HBAN', 132: 'TSCO', 119: 'MASI', 27: 'QRVO', 84: 'GOOGL', 23: 'SWKS', 
    110: 'TMUS', 182: 'UAL', 157: 'ADSK', 168: 'AMZN', 147: 'APA', 64: 'MKTX', 190: 'DXCM', 19: 'ALNY', 1: 'WING', 49: 'FITB', 194: 'PENN', 140: 'TXN', 133: 'ISRG', 
    177: 'SWAV', 32: 'NTAP', 22: 'ON', 77: 'VRTX', 104: 'WBA', 107: 'PODD', 59: 'Z', 72: 'ADI', 158: 'APLS', 169: 'MSFT', 94: 'PCTY', 66: 'LBRDK', 126: 'MU', 139: 'EXPE',
    159: 'STLD', 137: 'TTWO', 78: 'HOOD', 114: 'LPLA', 141: 'AMAT', 15: 'ABNB', 60: 'CRWD', 183: 'MCHP', 10: 'NDAQ', 135: 'DKNG', 197: 'SPLK', 99: 'PARA', 56: 'ETSY',
    13: 'TER', 62: 'RGEN', 80: 'TXG', 67: 'MRNA', 178: 'ZM', 39: 'CTAS', 173: 'FIVE', 184: 'DDOG', 162: 'ENPH', 16: 'ZBRA', 89: 'ENTG', 45: 'MSFT', 124: 'ASO', 42: 'SAIA',
    115: 'ILMN', 50: 'MTCH', 98: 'JBLU', 103: 'ZS', 40: 'CZR', 108: 'SEDG', 179: 'META', 6: 'POOL', 100: 'MQ', 48: 'WDAY', 150: 'PANW', 74: 'ALGN', 113: 'LULU', 163: 'COST',
    111: 'SPWR', 36: 'DLTR', 85: 'CAR', 79: 'WBD', 83: 'INTC', 75: 'CDNS', 57: 'IDXX', 180: 'GH', 93: 'ZION', 87: 'LSCC', 51: 'ROKU', 33: 'CROX', 58: 'ROP', 7: 'LRCX',
    172: 'APP', 61: 'LYFT', 185: 'ODFL', 102: 'TEAM', 188: 'RUN', 17: 'KLAC', 88: 'NFLX', 95: 'AMD', 14: 'ADBE', 54: 'SNPS', 18: 'ZI', 129: 'CFLT', 136: 'LITE', 191: 'TSLA',
    20: 'ULTA', 161: 'PTON', 5: 'OKTA', 71: 'EQIX', 34: 'REGN', 142: 'AVGO', 92: 'MSTR', 156: 'LCID', 41: 'NVDA', 69: 'SOFI', 138: 'SMCI', 174: 'AFRM', 11: 'COIN', 70: 'BYND',
    96: 'MRVL', 118: 'FCNCA', 29: 'ORLY', 143: 'TLRY', 86: 'ONEW', 82: 'OPEN', 127: 'MDB', 101: 'FCNCA', 8: 'BKNG', 31: 'NVCR'
}

In [5]:
yahoo_lst = []

In [6]:
for i in stock_dict.values():
    print(i)
    if i != 'ATVIX':
        yahoo_lst.append(pd.concat([get_book_data(i), get_prices(i)], axis=1))

KHC
KDP
MDLZ
CSCO
HOLX
MNST
EXC
CSX
GILD
CMCSA
INCY
SSNC
XEL
ATVIX
LNT
LKQ
AKAM
SBUX
FOXA
AEP
EBAY
SGEN
TXRH
DBX
PEP
FAST
ADP
FFIV
CTSH
EA
HAS
AGNC
VTRS
HON
HST
NBIX
CG
EXPD
PAYX
CINF
LSTR
JBHT
FANG
JKHY
AMGN
VRSK
PCAR
PTC
HTZ
GOOGL
CSGP
CDW
ROST
TECH
CPRT
CME
MIDD
UTHR
TROW
GEN
XRAY
PFG
BKR
PYPL
TRMB
AAL
MAR
FTNT
AXON
CHRW
DOCU
WDC
SBAC
CHK
HBAN
TSCO
MASI
QRVO
GOOGL
SWKS
TMUS
UAL
ADSK
AMZN
APA
MKTX
DXCM
ALNY
WING
FITB
PENN
TXN
ISRG
SWAV
NTAP
ON
VRTX
WBA
PODD
Z
ADI
APLS
MSFT
PCTY
LBRDK
MU
EXPE
STLD
TTWO
HOOD
LPLA
AMAT
ABNB
CRWD
MCHP
NDAQ
DKNG
SPLK
PARA
ETSY
TER
RGEN
TXG
MRNA
ZM
CTAS
FIVE
DDOG
ENPH
ZBRA
ENTG
MSFT
ASO
SAIA
ILMN
MTCH
JBLU
ZS
CZR
SEDG
META
POOL
MQ
WDAY
PANW
ALGN
LULU
COST
SPWR
DLTR
CAR
WBD
INTC
CDNS
IDXX
GH
ZION
LSCC
ROKU
CROX
ROP
LRCX
APP
LYFT
ODFL
TEAM
RUN
KLAC
NFLX
AMD
ADBE
SNPS
ZI
CFLT
LITE
TSLA
ULTA
PTON
OKTA
EQIX
REGN
AVGO
MSTR
LCID
NVDA
SOFI
SMCI
AFRM
COIN
BYND
MRVL
FCNCA
ORLY
TLRY
ONEW
OPEN
MDB
FCNCA
BKNG
NVCR


In [42]:
df_yahoo = pd.concat(yahoo_lst)
df_yahoo['stock_id'] = [i for i in stock_dict.values()if i != 'ATVIX']
df_yahoo = df_yahoo.reset_index(drop=True)
df_yahoo['stock_id'] = df_yahoo['stock_id'].astype(str)
df_yahoo = df_yahoo.set_index('stock_id')

In [43]:
df_yahoo

,operating_cash_flow_to_sales,free_cash_flow_to_net_income,cash_flow_to_capex,cash_dividends_paid_ratio,debt_to_equity_ratio,current_ratio,quick_ratio,gross_profit_margin,operating_income_perc,rdi_perc,eps,asset_turnover,return_on_assets,return_on_equity,dividends,lower_price_ratio,upper_price_ratio,std_to_mean_price_perc
stock_id,,,,,,,,,,,,,,,,,,
KHC,4.078740,3.011811,-3.822878,-1.937008,0.859042,1.004357,0.888519,0.340183,0.598210,0.000000,0.21,0.036640,0.002833,0.005267,1.60,0.859107,1.198493,0.068563
KDP,1.119691,0.884170,-4.754098,-0.538610,1.333874,0.386431,0.342296,0.554796,0.425391,0.000000,0.37,0.036754,0.010007,0.023355,1.03,0.851561,1.125563,0.051988
MDLZ,1.191296,0.902834,-4.129825,-0.532389,4.898610,0.608101,0.512727,0.386975,0.411276,0.000000,0.72,0.063710,0.013943,0.082244,1.62,0.878064,1.135919,0.055101
CSCO,0.651732,0.614898,-17.694030,-0.434305,1.415090,1.466950,0.783836,0.651554,0.460291,0.370880,0.90,0.074244,0.036829,0.088944,1.94,0.914030,1.167415,0.059069
HOLX,2.855408,2.275938,-4.927619,0.000000,17.941554,3.465999,3.284768,0.529250,0.377374,0.234029,0.37,0.051716,0.009913,0.187772,0.00,0.833372,1.143966,0.070152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OPEN,2.037736,2.141509,19.636364,-0.000000,-3.554693,51.608108,33.310811,0.097959,-0.822917,0.240000,-0.16,0.124145,-0.026856,0.068608,0.00,0.389576,2.046220,0.382292
MDB,-1.311465,-1.240537,-18.489894,-0.000000,14.317271,4.740132,1.530659,0.752717,-0.138750,0.345328,-0.41,0.080352,-0.010875,-0.166573,0.00,0.521453,1.403004,0.258045
FCNCA,1.602394,1.206117,-4.043624,-0.034574,0.720650,NaN,inf,0.000000,NaN,NaN,50.71,0.005939,0.003518,0.006053,3.89,0.432578,1.306798,0.239304


# Get data from Optiver

In [44]:
df = pd.read_csv('C:/Projects/smwap_project/data/train.csv')
df = df.drop(columns=['date_id', 'time_id', 'row_id', 'seconds_in_bucket'])
df = df[df['wap'].notna()]
df = df[df['far_price'].notna()]
df['stock_id'] = df['stock_id'].map(stock_dict)
df = df.groupby('stock_id').mean().reset_index().rename(columns={'imbalance_buy_sell_flag':'imb_flag', 'imbalance_size':'imb_size', 'reference_price':'ref_price'})
df['stock_id'] = df['stock_id'].astype(str)
df = df.set_index('stock_id')

In [45]:
df

,imb_size,imb_flag,ref_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target
stock_id,,,,,,,,,,,,
AAL,6.995861e+05,0.077974,1.000228,1.512112e+07,1.156660,1.000412,0.999879,359982.569039,1.000513,368893.675010,1.000197,-0.001134
ABNB,2.201408e+06,-0.004111,1.000055,5.127802e+07,1.000325,1.000031,0.999936,44814.971845,1.000165,46371.685490,1.000050,-0.198151
ADBE,6.933512e+06,-0.085080,1.000087,1.420875e+08,0.999602,0.999753,0.999957,55641.277674,1.000227,55447.698169,1.000089,0.067473
ADI,4.936751e+06,-0.097001,0.999878,9.599433e+07,0.997847,0.998126,0.999793,38078.649177,0.999970,40391.227505,0.999880,-0.055933
ADP,3.691595e+06,-0.119960,0.999868,7.186333e+07,0.998508,0.998799,0.999758,40165.563966,0.999999,43779.239880,0.999874,-0.133440
...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,7.846834e+05,-0.065357,1.000039,1.485423e+07,0.999473,0.999747,0.999664,40052.792567,1.000447,44916.922778,1.000049,-0.147732
ZI,1.076257e+06,-0.059448,1.000224,1.747520e+07,0.999271,0.999524,1.000047,48754.906745,1.000418,57698.029546,1.000226,-0.078326
ZION,5.170784e+05,-0.027448,0.999946,9.174938e+06,0.999097,0.999684,0.999822,31548.080899,1.000084,32518.790733,0.999949,0.584886


# Unite dataframes and export

In [46]:
df_final = df.join(df_yahoo, how='inner')

In [47]:
df_final

,imb_size,imb_flag,ref_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,...,operating_income_perc,rdi_perc,eps,asset_turnover,return_on_assets,return_on_equity,dividends,lower_price_ratio,upper_price_ratio,std_to_mean_price_perc
stock_id,,,,,,,,,,,,,,,,,,,,,
AAL,6.995861e+05,0.077974,1.000228,1.512112e+07,1.156660,1.000412,0.999879,359982.569039,1.000513,368893.675010,...,0.270190,0.000000,-0.83,0.102586,0.000000,-0.000000,0.00,0.744960,1.308825,0.127075
ABNB,2.201408e+06,-0.004111,1.000055,5.127802e+07,1.000325,1.000031,0.999936,44814.971845,1.000165,46371.685490,...,0.509190,0.290569,6.83,0.079225,0.204021,-0.820946,0.00,0.676509,1.243477,0.107683
ADBE,6.933512e+06,-0.085080,1.000087,1.420875e+08,0.999602,0.999753,0.999957,55641.277674,1.000227,55447.698169,...,NaN,NaN,3.26,NaN,0.048230,0.241938,0.00,0.684283,1.361457,0.205689
ADI,4.936751e+06,-0.097001,0.999878,9.599433e+07,0.997847,0.998126,0.999793,38078.649177,0.999970,40391.227505,...,0.454511,0.452643,1.00,0.027836,0.010215,0.015929,3.44,0.860918,1.126320,0.053240
ADP,3.691595e+06,-0.119960,0.999868,7.186333e+07,0.998508,0.998799,0.999758,40165.563966,0.999999,43779.239880,...,0.509664,0.000000,2.09,0.043698,0.017424,-1.065460,5.15,0.888491,1.132731,0.060665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,7.846834e+05,-0.065357,1.000039,1.485423e+07,0.999473,0.999747,0.999664,40052.792567,1.000447,44916.922778,...,0.112412,0.335092,-0.28,0.065203,-0.002046,-0.011103,0.00,0.719524,1.300608,0.125880
ZI,1.076257e+06,-0.059448,1.000224,1.747520e+07,0.999271,0.999524,1.000047,48754.906745,1.000418,57698.029546,...,0.253532,0.235558,0.08,0.022190,0.004271,0.006925,0.00,0.579239,1.455127,0.220495
ZION,5.170784e+05,-0.027448,0.999946,9.174938e+06,0.999097,0.999684,0.999822,31548.080899,1.000084,32518.790733,...,NaN,NaN,1.13,0.004383,0.002005,0.002120,1.64,0.518761,1.568215,0.225722


In [52]:
df_final.reset_index().sort_index(axis=1).to_csv('C:/Projects/smwap_project/data/data.csv', index=False)